In [1]:
import MetaTrader5 as mt5
import numpy as np
import pandas as pd
import warnings
from librerias_estrategias import Estrategia_SuperTrend
warnings.filterwarnings("ignore")
# pd.set_option('display.max_rows', None)

import matplotlib.dates as mpl_dates
import matplotlib.pyplot as plt

from datetime import datetime 
import schedule
import time

In [2]:
# usuario = 
# contraseña =  
# mt5.login(usuario, contraseña)
if not mt5.initialize():
    print('Error al iniciar: ',mt5.last_error())
    quit()
else:
    print('Inicio correctamente')

Inicio correctamente


In [3]:
st = Estrategia_SuperTrend

# Importando Datos MT5

https://www.mql5.com/es/docs/python_metatrader5

In [4]:
def Simbolo(divisa, tf,em,ter):  
           
    temp = {'M1':mt5.TIMEFRAME_M1,'M5':mt5.TIMEFRAME_M5,
            'M15':mt5.TIMEFRAME_M15,'M30':mt5.TIMEFRAME_M30,
            'H1':mt5.TIMEFRAME_H1,'H4':mt5.TIMEFRAME_H4
           }
    for clave, cont in temp.items():
        if tf == clave:
            rates = mt5.copy_rates_from_pos(divisa, cont, em, ter)

    # Transform Tuple into a DataFrame
    df_rates = pd.DataFrame(rates).dropna()
    

    # Convert number format of the date into date format
    df_rates["time"] = pd.to_datetime(df_rates["time"], unit="s")
    df_rates = df_rates.drop(['tick_volume','spread','real_volume'],axis=1)
    
    df_rates.time = pd.to_datetime(df_rates['time'])
    df_rates.index = df_rates.time
    df_rates.index.name = 'date'
    df_rates['time'] = df_rates['time'].apply(mpl_dates.date2num)

    return df_rates

symbol    = 'NZDCAD'#'EURUSD' 
timeframe = 'M5'
ini = 0
ter = 180
df = Simbolo(symbol,timeframe,ini,ter)

# SuperTrend (ATR)

https://medium.datadriveninvestor.com/the-supertrend-implementing-screening-backtesting-in-python-70e8f88f383d

In [5]:
velas_normales = st.SuperTrend(df)

dt = st.Heikin_Ashi(df)
velas_HA = st.SuperTrend(dt)

#  Manejo de ordenes

* Documentacion:https://www.mql5.com/es/docs/python_metatrader5

In [7]:
def orders(symbol, lot, buy=False):
    
    """ Send the orders """

    filling_mode = mt5.symbol_info(symbol).filling_mode
    deviation = 10
    
    # **************************** Open a trade *****************************      
    # Buy order Parameters
    if buy:
        type_trade = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
        sl = price - 50 * mt5.symbol_info(symbol).point
#             sl = mt5.symbol_info_tick(symbol).ask * (100-0.01)
        tp = price + 100 * mt5.symbol_info(symbol).point 
#             tp = mt5.symbol_info_tick(symbol).ask * (300+0.01)

    # Sell order Parameters
    else:            
        type_trade = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
        sl = price + 50 * mt5.symbol_info(symbol).point
#             sl = price * (1+0.01)
        tp = price - 100 * mt5.symbol_info(symbol).point 
#             tp = price * (3-0.01)

    # Open the trade
    request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": symbol,
                "volume": lot,
                "type": type_trade,
                "price": price,
                "deviation": deviation,
                "sl": sl,
                "tp": tp,
                "magic": 234000,
                "comment": "python script order",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": filling_mode,
                }
    
    # send a trading request
    result = mt5.order_send(request)
    
    return result.comment

In [8]:
posicion = False 

def señales_compra_o_venta(data, symbol):
    global posicion
      
    print('buscando señales..........','\n\n')
    
    actual    = len(data.index) - 1
    anterior = actual - 1
    
    if data['SuperTrend'][anterior] == True and data['SuperTrend'][actual]==False:
        print('Cambio de tendencia, VENTAS','\n')
        if not posicion:
            print(orders(symbol, 0.01, False))
            posicion = True
        else:
            print('Ya se encuentra en posicion de VENTAS')
           
    if data['SuperTrend'][anterior] == False and data['SuperTrend'][actual] == True:
        print('Cambio de tendencia, COMPRAS','\n')
        if posicion:
            print(orders(symbol, 0.01, True))
            posicion = False
        else:
            print('Ya se encuentra en posicion de COMPRAS')

# Bot de negociacion

In [9]:
def run_bot():
    data   = Simbolo(symbol,timeframe,0,100)
    print(f'Buscando barras nuevas desde: {data.index[-1]}','\n')
    velas_HA = st.Heikin_Ashi(data)
    STdata = st.SuperTrend(velas_HA)
    señales_compra_o_venta(STdata, symbol)

schedule.every(10).seconds.do(run_bot)

while True:
    schedule.run_pending()
    
    time.sleep(1) 

Buscando barras nuevas desde: 2023-05-09 01:10:00 

buscando señales.......... 


Buscando barras nuevas desde: 2023-05-09 01:10:00 

buscando señales.......... 


Buscando barras nuevas desde: 2023-05-09 01:10:00 

buscando señales.......... 


Buscando barras nuevas desde: 2023-05-09 01:15:00 

buscando señales.......... 


Buscando barras nuevas desde: 2023-05-09 01:15:00 

buscando señales.......... 


Buscando barras nuevas desde: 2023-05-09 01:15:00 

buscando señales.......... 


Buscando barras nuevas desde: 2023-05-09 01:15:00 

buscando señales.......... 


Buscando barras nuevas desde: 2023-05-09 01:15:00 

buscando señales.......... 


Buscando barras nuevas desde: 2023-05-09 01:15:00 

buscando señales.......... 


Buscando barras nuevas desde: 2023-05-09 01:15:00 

buscando señales.......... 


Buscando barras nuevas desde: 2023-05-09 01:15:00 

buscando señales.......... 


Buscando barras nuevas desde: 2023-05-09 01:15:00 

buscando señales.......... 


Buscando barras 

KeyboardInterrupt: 

In [ ]:
# Create the bound between MT5 and Python
mt5.shutdown()